# Gastebois Jacques - Ei2i
## Visualisation des donnees synthetiques et des trajectoires

Le ressenti est une valeur moins quantifiable mais primordiale. Ici, on va essayer de voir ce qu'on a reellement appris a la machine.

L'idee c'est d'afficher nos images distillées pour verifier qu'elles ne sont pas juste du bruit, mais qu'elles capturent bien l'essence des classes de CIFAR-10.

---

### 1. imports pour la partie graphique

on sort matplotlib et torchvision pour faire de jolies grilles. j'utilise des noms de variables un peu a ma sauce comme d'hab.

In [ ]:
import torch
import matplotlib.pyplot as plt
import  numpy as np 
from torchvision.utils import make_grid
import os

# pour que ca s'affiche bien dans le notebook
%matplotlib inline

def denormalize(tensor, mean, std):
    # petite fonction pour remettre les couleurs d'equerre
    for t, m,  s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

### 2. recuperation des resultats

on va chercher les fichiers .pt qu'on a du sauvegarder pendant la distillation. si t'as pas lance le premier notebook ca risque de faire un bide :/

In [ ]:
run_path =  './logged_files/CIFAR10/ton_run_ici' # a changer avec le vrai nom du dossier
img_file = 'images_best.pt'

try :
    # on charge le bouzin
    img_syn =  torch.load(os.path.join(run_path, img_file), map_location='cpu')
    print(f"images chargees : {img_syn.shape} :)")
except :
    print("chemin pas bon ou fichier manquant. verifie logged_files/ :(")
    img_syn = None

### 3. affichage de la grille

on affiche une grille avec 10 images par ligne (une par classe pour IPC=1). c'est la qu'on voit si on a fait du bon boulot ou si c'est de la bouillie de pixels.

In [ ]:
if img_syn is not  None:
    # on remet les couleurs dans le bon sens (les stats de cifar10)
    stats_m = [0.4914, 0.4822, 0.4465] 
    stats_s = [0.2023, 0.1994, 0.2010]
    
    grid_img = make_grid(img_syn, nrow=10, normalize=False) 
    grid_img  = denormalize(grid_img, stats_m, stats_s)
    
    plt.figure(figsize=(12,  6))
    plt.imshow(grid_img.permute(1, 2, 0).numpy().clip(0, 1))
    plt.axis( 'off')
    plt.title( "images distillees - cifar10 (ipc=1) :p")
    plt.show()
else:
	print( "rien a afficher l'ami :(")

### 4. analyse des trajectoires (pca)

cette partie la est plus complexe elle demande d'avoir sauve les trajectoires du student. j'ai un petit script pour ca mais dans le notebook on peut juste verifier que les fichiers sont la.

In [ ]:
trajectory_dir = os.path.join(run_path, "trajectories")
if os.path.exists(trajectory_dir):
    fichiers = os.listdir(trajectory_dir)
    print(f"on a trouve {len(fichiers)} etapes de trajectoires. c'est bon pour la pca ca :)")
else :
    print("pas de trajectoires ici. t'as pas du mettre save_trajectory_interval dans la config :/")